In [36]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import kagglehub
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import models
from PIL import Image

In [37]:
path = kagglehub.dataset_download("atulyakumar98/pothole-detection-dataset")

normal_dir = os.path.join(path, 'normal')
potholes_dir = os.path.join(path, 'potholes')
print("Normal images path:", normal_dir)
print("Potholes images path:", potholes_dir)

Normal images path: /root/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/normal
Potholes images path: /root/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/potholes


In [38]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /root/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4


In [39]:
class PotholeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_path).convert("RGB")

        label = 0 if 'normal' in self.image_dir else 1

        if self.transform:
            image = self.transform(image)

        return image, label

In [40]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


normal_dataset = PotholeDataset(image_dir=normal_dir, transform=transform)
potholes_dataset = PotholeDataset(image_dir=potholes_dir, transform=transform)

combined_dataset = normal_dataset + potholes_dataset

train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(combined_dataset, batch_size=32, shuffle=False)

In [41]:
# MobileNetV2 Model for Classification
mobilenet_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet_model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(mobilenet_model.last_channel, 2),
)

In [42]:
#ResNet50 Model for Classification
resnet_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2) # Match the number of classes

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet_model = mobilenet_model.to(device)
resnet_model = resnet_model.to(device)

In [44]:
criterion = nn.CrossEntropyLoss()
mobilenet_optimizer = optim.Adam(mobilenet_model.classifier.parameters(), lr=0.001)
resnet_optimizer = optim.Adam(resnet_model.fc.parameters(), lr=0.001)

In [45]:
def train_classification_model(model, optimizer, train_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        runningLoss = 0.0
        correctPredictions = 0.0
        totalPredictions = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

        epochLoss = runningLoss / len(train_loader)
        epochAccuracy = (correctPredictions / totalPredictions) * 100
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epochLoss:.4f}, Accuracy: {epochAccuracy:.2f}%")

In [46]:
def evaluate_classification_model(model, test_loader):
    model.eval()
    correctPredictions = 0.0
    totalPredictions = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

    accuracy = (correctPredictions / totalPredictions) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

In [47]:
# Choose model and train
print("Training MobileNetV2...")
train_classification_model(mobilenet_model, mobilenet_optimizer, train_loader)
evaluate_classification_model(mobilenet_model, test_loader)

print("Training ResNet50...")
train_classification_model(resnet_model, resnet_optimizer, train_loader)
evaluate_classification_model(resnet_model, test_loader)

Training MobileNetV2...
Epoch 1/10, Loss: 0.3815, Accuracy: 85.61%
Epoch 2/10, Loss: 0.1947, Accuracy: 92.80%
Epoch 3/10, Loss: 0.1286, Accuracy: 97.65%
Epoch 4/10, Loss: 0.1342, Accuracy: 95.45%
Epoch 5/10, Loss: 0.1162, Accuracy: 96.33%
Epoch 6/10, Loss: 0.1410, Accuracy: 95.59%
Epoch 7/10, Loss: 0.0972, Accuracy: 97.50%
Epoch 8/10, Loss: 0.0949, Accuracy: 97.06%
Epoch 9/10, Loss: 0.0839, Accuracy: 97.50%
Epoch 10/10, Loss: 0.0885, Accuracy: 97.06%
Test Accuracy: 98.97%
Training ResNet50...
Epoch 1/10, Loss: 0.4192, Accuracy: 91.19%
Epoch 2/10, Loss: 0.1852, Accuracy: 97.50%
Epoch 3/10, Loss: 0.1270, Accuracy: 97.94%
Epoch 4/10, Loss: 0.1107, Accuracy: 98.68%
Epoch 5/10, Loss: 0.0994, Accuracy: 97.80%
Epoch 6/10, Loss: 0.0892, Accuracy: 97.36%
Epoch 7/10, Loss: 0.0739, Accuracy: 98.83%
Epoch 8/10, Loss: 0.0826, Accuracy: 98.53%
Epoch 9/10, Loss: 0.0721, Accuracy: 98.38%
Epoch 10/10, Loss: 0.0612, Accuracy: 98.68%
Test Accuracy: 98.83%
